# **Validation Suite**

## Purpose and Setup
This notebook will allow you to test the output from WRF-GHG against surface, upper air, and satelite observations.

The next cell imports modules needed to work properly

In [ ]:
import wrf
import numpy as np
import pandas as pd
from datetime import datetime as dt
from netCDF4 import Dataset #type: ignore
from termcolor import cprint
from numpy import unravel_index
import collections.abc as c

### *Base class*

This class is used for all the following classes in order to give some basic location data to each class.

In [ ]:
class Base_point:
    '''
    Base_point: parent class for validation. Sets location name.
    '''
    def __init__(self: object, loc: str, **kwargs) -> None:
        self.loc = loc

### *Upper air (UA) class*

This class is used to set the attributes for any data objects that cares about upper air data. This is usually the WRF data that you're testing or the radiosonde data you use for validation. It inherits from the base class.

In [ ]:
class UA_point(Base_point):
    def __init__(self: object, loc: str, **kwargs) -> None:
        super().__init__(loc, **kwargs)
    def __eq__(self: object, other: object) -> bool:
        try:
            results: np.ndarray[bool] = np.empty(5, bool)
            results[0] = np.allclose(self.p, other.p, atol=10.)
            results[1] = np.allclose(self.t, other.t, atol=1.)
            results[2] = np.allclose(self.td, other.td, atol=1.)
            results[3] = np.allclose(self.wdir, other.wdir, atol=5.)
            results[4] = np.allclose(self.wspd, other.wspd, atol=1.)
            result: bool = results.all()
        except AttributeError:
            if isinstance(self, WRF_point):
                result: bool = other.__eq__(self)
            else:
                raise NotImplementedError('Compairison not implemented')
        finally:
            return result

### *Satelite (Sat) class*

This class is used to set the attributes for any data objects that cares about satelite data. This is usually the WRF data or TROPOMI or OCO-2 data. It inherits from the base class. **Note**: The respective classes for TROPOMI and OCO-2 are still in development (9-9-2024)

In [ ]:
class Sat_point(Base_point):
    def __init__(self: object, loc: str, **kwargs) -> None:
        super().__init__(loc, **kwargs)
    def sat_loc(self: object, ulat: float, ulon: float, lats: c.Iterable[float], lons: c.Iterable[float]) -> tuple[int] | int:
        R = 6371000
        lat1 = np.radians(ulat)
        lat2 = np.radians(lats)
        delta_lat = np.radians(lats-ulat)
        delta_lon = np.radians(lons-ulon)
        a = (np.sin(delta_lat/2))*(np.sin(delta_lat/2))+(np.cos(lat1))*(np.cos(lat2))*(np.sin(delta_lon/2))*(np.sin(delta_lon/2))
        c = 2*np.arctan2(np.sqrt(a),np.sqrt(1-a))
        d = R*c
        if d.ndim == 1:
            return d.argmin()
        else:
            x, y = unravel_index(d.argmin(),d.shape)
            return x,y
    def __eq__(self: object, other: object) -> bool:
        result: bool | None = None
        try:
            if isinstance(self, Tropomi_point) or (isinstance(self, WRF_point) and isinstance(other, Tropomi_point)):
                results: np.ndarray[bool] = np.empty(2, bool)
                results[0] = np.abs(self.xch4 - other.xch4) <= 1.e-1
                results[1] = np.abs(self.xco - other.xco) <= 1.e-1
                result = results.all()
            #! elif for OCO-2 here
        except AttributeError:
            if isinstance(self, WRF_point):
                result = other.__eq__(self)
            else:
                result = NotImplemented
        else:
            if result is None:
                if isinstance(self,WRF_point):
                    result = other.__eq__(self)
                else:
                    raise NotImplementedError('Compairison not implemented')
            else:
                pass
        finally:
            return result

### *Surface class*

This class is used to set the attirbutes for any data object that cares about surface data. This is usually the WRF data or the ASOS observation data. It inherits from the base class.

In [ ]:
class Surface_point(Base_point):
    def __init__(self, loc, **kwargs):
        super().__init__(loc, **kwargs)
    def __eq__(self, other: object) -> bool:
        try:
            results = np.empty(5, bool)
            results[0] = abs(self.T2 - other.T2) <= 0.5
            results[1] = abs(self.td2 - other.td2) <= 0.5
            results[2] = abs(self.slp - other.p) <= 3.0
            results[3] = abs(self.wspd10 - other.wspd10) <= 0.2
            results[4] = abs(self.wdir10 - other.wdir10) <= 5
            result = results.all()
        except AttributeError:
            if isinstance(self, WRF_point):
                result = other.__eq__(self)
            else:
                result = NotImplemented
        finally:
            return result